In [8]:
using Sunny, GLMakie

units = Units(:meV, :angstrom);

In [10]:
latvecs = Dict()
latvecs["Ni"] = lattice_vectors(10.02, 5.86, 4.68, 90, 90, 90)
latvecs["Fe"] = lattice_vectors(10.337, 6.011, 4.695, 90, 90, 90)

3×3 StaticArraysCore.SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 10.337  0.0    0.0
  0.0    6.011  0.0
  0.0    0.0    4.695

We continue from here with the Ni system

In [11]:
cryst = Crystal(latvecs["Ni"], [[1/4, 1/4, 0]], 62, types = ["Ni"])
sys = System(cryst, [1 => Moment(s=1, g=2)], :dipole_uncorrected)

System [Dipole mode, large-s]
Supercell (1×1×1)×4
Energy per site 0


In [ ]:
Jbc =  1.036
Jb  =  0.6701
Jc  = -0.0469
Jac = -0.1121
Jab =  0.2977
Da  =  0.1969
Db  =  0.9097
Dc  =  0.0
set_exchange!(sys, Jbc, Bond(2, 3, [0, 0, 0]))
set_exchange!(sys, Jc , Bond(1, 1, [0, 0,-1]))
set_exchange!(sys, Jb , Bond(1, 1, [0, 1, 0]))
set_exchange!(sys, Jab, Bond(1, 2, [0, 0, 0]))
set_exchange!(sys, Jab, Bond(3, 4, [0, 0, 0]))
set_exchange!(sys, Jac, Bond(3, 1, [0, 0, 0]))
set_exchange!(sys, Jac, Bond(4, 2, [0, 0, 0]))
set_onsite_coupling!(sys, S -> Da*S[1]^2 + Db*S[2]^2 + Dc*S[3]^2, 1)

In [19]:
randomize_spins!(sys)
minimize_energy!(sys)
plot_spins(sys; color=[S[3] for S in sys.dipoles])

In [35]:
qs = [[ 0,0,0], [0,2,0]]
labels = ["a", "b"]  # TODO: Might not be correct labels!
path = q_space_path(cryst, qs, 600; labels)

QPath (600 samples)
  a → b


In [37]:
kernel = lorentzian(fwhm=0.4)
energies = range(0.0, 12.0, 150)
swt = SpinWaveTheory(sys; measure=ssf_perp(sys))
res = intensities(swt, path; energies, kernel)
plot_intensities(res)

## 2: Inhomogenous system

In [ ]:
sys_inhom = to_inhomogeneous(repeat_periodically(sys, (4, 4, 4))) # TODO: change to 10 10 10

System [Dipole mode, large-s]
Supercell (10×10×10)×4
Energy per site -2.268


In [47]:
for (site1, site2, offset) in symmetry_equivalent_bonds(sys_inhom, Bond(1, 1, [1, 0, 0]))
	noise = 0.0
	set_exchange_at!(sys_inhom, 1.0 + noise, site1, site2; offset)
end

randomize_spins!(sys_inhom)
minimize_energy!(sys_inhom, maxiters=5_000)
plot_spins(sys_inhom; color=[S[3] for S in sys_inhom.dipoles], ndims=3)

┌ Warning: Non-converged after 5000 iterations: |ΔE|=1.8e-12, |Δx|=2e-07, |∂E/∂x|=5.5e-07
└ @ Sunny C:\Users\45267\.julia\packages\Sunny\NXlhu\src\Optimization.jl:156


In [ ]:
swt = SpinWaveTheoryKPM(sys_inhom; measure=ssf_perp(sys_inhom), tol=0.05)
res = intensities(swt, path; energies, kernel)
plot_intensities(res)